# Omission LFP Analysis

Brief 1-2 sentence description of notebook.

In [21]:
import os
import glob

In [1]:
# Imports of all used packages and libraries
import pandas as pd

In [26]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

## Inputs & Data

Explanation of each input and where it comes from.

In [77]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
INPUT_VARIABLE = 1
CHANNEL_MAPPING_DF = pd.read_excel("./channel_mapping.xlsx")
TONE_TIMESTAMP_DF = pd.read_excel("./tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TIME_HALFBANDWIDTH_PRODUCT = 1
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.1
BRAIN_REGION_COL = ["spike_interface_mPFC",
"spike_interface_vHPC",
"spike_interface_BLA",
"spike_interface_LH",
"spike_interface_MD"]
TRIAL_TIME_STAMP_DURATION = 1000*10

In [34]:
all_session_dir = ["/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec", "/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec"]

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,1.1,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
1,1,1.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,1.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,1.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,NaN,NaN,NaN,NaN,NaN
7,2,1.4,NaN,15,14,13,12,2.0,31.0,30.0,29.0,28.0


In [57]:
cohort_2_channel_mapping = CHANNEL_MAPPING_DF[CHANNEL_MAPPING_DF["Cohort"] == 2]

In [58]:
cohort_2_channel_mapping

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,NaN,NaN,NaN,NaN,NaN
7,2,1.4,NaN,15,14,13,12,2.0,31.0,30.0,29.0,28.0


In [13]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="condition")

In [14]:
all_trials_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
1643,5681112,1,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,2488068,20230621_111240_standard_comp_to_omission_D5_s...,2483,1,1-4_t3b3L_box1,1.4,Subj 2 Only,NaN
1644,8081141,1,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,4888097,20230621_111240_standard_comp_to_omission_D5_s...,4879,1,1-4_t3b3L_box1,1.2,Subj 2 then Subj 1,NaN
1645,10381173,1,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,7188129,20230621_111240_standard_comp_to_omission_D5_s...,7175,1,1-4_t3b3L_box1,1.2,Subj 1 then Subj 2,NaN
1646,12181192,1,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,8988148,20230621_111240_standard_comp_to_omission_D5_s...,8972,1,1-4_t3b3L_box1,1.2,Subj 1 Only,NaN
1647,13281209,1,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,10088165,20230621_111240_standard_comp_to_omission_D5_s...,10069,1,1-4_t3b3L_box1,1.2,Subj 1 Only,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,37719707,1,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,dio_ECU_Din1,36077451,20230619_115321_standard_comp_to_omission_D3_s...,36009,4,1-4_t3b3L_box2,rewarded,NaN,NaN
2020,38819710,1,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,dio_ECU_Din1,37177454,20230619_115321_standard_comp_to_omission_D3_s...,37107,4,1-4_t3b3L_box2,rewarded,NaN,NaN
2021,40119707,1,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,dio_ECU_Din1,38477451,20230619_115321_standard_comp_to_omission_D3_s...,38405,4,1-4_t3b3L_box2,rewarded,NaN,NaN
2022,41219709,1,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,dio_ECU_Din1,39577453,20230619_115321_standard_comp_to_omission_D3_s...,39503,4,1-4_t3b3L_box2,rewarded,NaN,NaN


In [35]:
recording_name_to_object = {}
for session_dir in all_session_dir:
    for recording_path in glob.glob(os.path.join(session_dir, "*merged.rec")):
        recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
        print(recording_basename)

        current_recording = se.read_spikegadgets(recording_path, stream_id="trodes")
        current_recording = sp.bandpass_filter(current_recording, freq_min=0.5, freq_max=300)
        current_recording = sp.notch_filter(current_recording, freq=60)
        current_recording = sp.resample(current_recording, resample_rate=1000)
        recording_name_to_object[recording_basename] = current_recording



20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged


In [42]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_object.keys())].reset_index(drop=True)

In [43]:
all_trials_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
0,8260195,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,5757314,20230620_114347_standard_comp_to_omission_D4_s...,5745,2,1-1_t1b2L_box_2,1.2,Subj 2 blocking Subj 1,NaN
1,10060217,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,7557336,20230620_114347_standard_comp_to_omission_D4_s...,7542,2,1-1_t1b2L_box_2,1.1,Subj 1 Only,NaN
2,11160233,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,8657352,20230620_114347_standard_comp_to_omission_D4_s...,8640,2,1-1_t1b2L_box_2,1.1,Subj 1 then Subj 2,1.1 still at port at 8791
3,12160245,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,9657364,20230620_114347_standard_comp_to_omission_D4_s...,9638,2,1-1_t1b2L_box_2,1.1,Subj 1 Only,NaN
4,14060266,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,11557385,20230620_114347_standard_comp_to_omission_D4_s...,11535,2,1-1_t1b2L_box_2,1.1,Subj 1 Only,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,61534911,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,60699231,20230618_100636_standard_comp_to_omission_D2_s...,60585,2,1_4_t4b3L_box1,rewarded,NaN,NaN
300,63034911,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,62199231,20230618_100636_standard_comp_to_omission_D2_s...,62082,2,1_4_t4b3L_box1,rewarded,NaN,NaN
301,65034907,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,64199227,20230618_100636_standard_comp_to_omission_D2_s...,64078,2,1_4_t4b3L_box1,omission,NaN,NaN
302,66934914,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,66099234,20230618_100636_standard_comp_to_omission_D2_s...,65974,2,1_4_t4b3L_box1,rewarded,NaN,NaN


In [65]:
cohort_2_channel_mapping["Subject"]

4    1.1
5    1.2
6    1.3
7    1.4
Name: Subject, dtype: float64

In [74]:
for recording_name, lfp_signal in recording_name_to_object.items():
    print(recording_name)
    current_subject = float(".".join(recording_name.split("subj")[-1].strip("merged").strip("_").replace("-","_").split("_")[:2]))
    current_channel_map = cohort_2_channel_mapping[cohort_2_channel_mapping["Subject"] == current_subject].iloc[0]
    current_trial_df = all_trials_df[(all_trials_df["recording_file"] == recording_name) & all_trials_df["video_number"] == 1].reset_index(drop=True)
    for brain_region in brain_region_col:
        current_channel = str(int(current_channel_map[brain_region]))
        print(current_channel)
        for index, row in current_trial_df.iterrows():
            pass
            break
        break
    break

20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
2


In [70]:
current_channel_map

Cohort                   2.0
Subject                  1.2
eib_mPFC                 NaN
eib_vHPC                31.0
eib_BLA                 30.0
eib_LH                  29.0
eib_MD                  28.0
spike_interface_mPFC    10.0
spike_interface_vHPC    31.0
spike_interface_BLA     30.0
spike_interface_LH      29.0
spike_interface_MD      28.0
Name: 5, dtype: float64

In [61]:
current_trial_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
0,8260195,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,5757314,20230620_114347_standard_comp_to_omission_D4_s...,5755,1,1-2_t3b3L_box_1,1.2,Subj 2 blocking Subj 1,NaN
1,10060217,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,7557336,20230620_114347_standard_comp_to_omission_D4_s...,7552,1,1-2_t3b3L_box_1,1.1,Subj 1 Only,NaN
2,11160233,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,8657352,20230620_114347_standard_comp_to_omission_D4_s...,8650,1,1-2_t3b3L_box_1,1.1,Subj 1 then Subj 2,1.1 still at port at 8791
3,12160245,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,9657364,20230620_114347_standard_comp_to_omission_D4_s...,9648,1,1-2_t3b3L_box_1,1.1,Subj 1 Only,NaN
4,14060266,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,11557385,20230620_114347_standard_comp_to_omission_D4_s...,11544,1,1-2_t3b3L_box_1,1.1,Subj 1 Only,NaN
5,15560284,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,13057403,20230620_114347_standard_comp_to_omission_D4_s...,13042,1,1-2_t3b3L_box_1,1.2,Subj 2 blocking Subj 1,NaN
6,17660313,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,15157432,20230620_114347_standard_comp_to_omission_D4_s...,15138,1,1-2_t3b3L_box_1,1.1,Subj 1 Only,NaN
7,18660323,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,16157442,20230620_114347_standard_comp_to_omission_D4_s...,16136,1,1-2_t3b3L_box_1,1.1,Subj 2 then Subj 1,1.2 still at port until 16194
8,20260345,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,17757464,20230620_114347_standard_comp_to_omission_D4_s...,17733,1,1-2_t3b3L_box_1,1.2,Subj 2 then Subj 1,1.2 at port unitl 17848
9,21560358,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,19057477,20230620_114347_standard_comp_to_omission_D4_s...,19030,1,1-2_t3b3L_box_1,1.1,Subj 1 Only,NaN


In [38]:
condition_to_brain_region = nested_dict()

In [39]:
for index, row in tone_df.iterrows():
    for brain_region, channel_id in subj_1_1_brain_region_to_channel.items():
        current_trace = subj_1_1_lfp.get_traces(channel_ids=[channel_id], start_frame=row["lfp_index"], end_frame=row["lfp_index"] + 1000 * 10)
        condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["traces"] = current_trace
        
                # Compute multitaper spectral estimate
        m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=current_trace,
                       sampling_frequency=resampled_frequency)
        
        # Sets up computing connectivity measures/power from multitaper spectral estimate
        c = Connectivity.from_multitaper(m)
        
        if condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"]:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"].append(c.frequencies)
        else:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"] = [c.frequencies]

        if condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"]:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"].append(c.power().squeeze())
        else:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"] = [c.power().squeeze()]
